In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#train = pd.read_csv("./open/sample_train.csv")
train = pd.read_csv("./open_code_train_220602_수정.csv")
train.head(5)

,Unnamed: 0,code1,c1_problem_name,code2,c2_problem_name,similar
0,0,from collections import deque\nmod = 10**9+7\n...,145,"from collections import deque\nN, M = [int(x) ...",145,1
1,1,"def selection_sort(numbers, n):\ncounter = 0\n...",15,"n,x,t = map(int,(input().split()))\ni = 1\nwhi...",86,0
2,2,"n,k,*a = map(int,open(0).read().split())\ndef ...",99,n = int(input())\nd = {}\nt = 0\nfor i in rang...,218,0
3,3,N = str(input())\nprint(chr(ord(N)+1)),239,k = int(input())\ns = input()\nif len(s)>k:\np...,143,0
4,4,"n, a, b = list(map(int, input().split()))\np =...",214,import sys\ninput = sys.stdin.buffer.readline\...,74,0


In [3]:
test = pd.read_csv("./test_220602.csv")
test.head(5)

,Unnamed: 0,pair_id,code1,code2
0,0,1,def main():\ns = input()\nif s.count('a') == 1...,"N,K = map(int,input().split())\nA = list(map(i..."
1,1,2,"N,K,Q = map(int,input().split())\npoints = [0]...","N, K, Q = map(int,input().split())\n\nif K > Q..."
2,2,3,from itertools import combinations\nn = int(in...,s = input()\nt = input()\nlength_s = len(s)\nl...
3,3,4,"a,b=map(int,input().split())\n\nans1=a+b\nans2...","a, b, c, d = map(int,input().split())\n\nif a ..."
4,4,5,S = input()\nK = int(input())\n\nind = -1\nfor...,"H, W = map(int, input().split())\ngrid = []\nf..."


In [4]:
model = CountVectorizer()

In [5]:
model.fit(train['code1'])
model.fit(train['code2'])

CountVectorizer()

In [6]:
code1_vecs = model.transform(test['code1'])
code2_vecs = model.transform(test['code2'])

In [7]:
from tqdm import tqdm

In [8]:
preds = []
for code1_vec, code2_vec in tqdm(zip(code1_vecs, code2_vecs)) :
    preds.append(cosine_similarity(code1_vec, code2_vec))

179700it [03:12, 932.59it/s] 


In [9]:
preds_result = np.reshape(preds, len(preds))

In [10]:
preds_result

array([0.13363062, 0.83402338, 0.24807103, ..., 0.2045983 , 0.80984121,
       0.49008412])

In [13]:
final = np.where(preds_result > 0.5, 1, 0)

In [14]:
final

array([0, 1, 0, ..., 0, 1, 0])

In [15]:
df = pd.DataFrame(final)
df.value_counts()

0    117862
1     61838
dtype: int64

In [16]:
df.value_counts()[1] / df.value_counts()[0] * 100

52.46644380716431

In [17]:

submission = pd.read_csv('./sample_submission.csv')
submission['similar'] = final
submission.to_csv('./submission.csv', index=False)
